In [1]:
import json
import datetime
import time
import requests
import nbimporter
import random
import os
import psutil
import psycopg2
from multiprocessing import Process, Pool
import Insert_py
import Get_py
import Proxy_Gatherer as pg
import Init_database as indb

Importing Jupyter notebook from Insert_py.ipynb
Importing Jupyter notebook from Get_py.ipynb
Importing Jupyter notebook from Proxy_Gatherer.ipynb
Importing Jupyter notebook from Init_database.ipynb


In [2]:
# #--------THIS ONLY RUNS ONE TIME TO CREATE A DATABASE FRAMEWORK--------#
# import Create_Teld_Database as ctd
# db_init = ctd.create_teld_database()

In [2]:
#--------INITIALIZING CONNECTION TO DATABASE--------#
db_name="db_teld"
user_name="postgres"
password = "postgres"
host="127.0.0.1"
port="5432"
conn = indb.init_db(db_name, user_name, password, host, port)

In [3]:
##------- Specify request header ------- ##
hdr = {
    'accept': "*/*",
    'user-agent': "Teld/3.4.0 (iPhone; iOS 10.3.3; Scale/2.00)",
    'cookie': "TELDAppID=",
    'content-length': "111",
    'connection': "keep-alive",
    'accept-encoding': "gzip, deflate",
    'device': "app_version=3.4.0&os_version=10.3.3&client=ios&device_name=iPhone 6s (A1633/A1688/A1691/A1700)&device_id=38CD294C-CC36-4998-9074-C7F798EE6087&city_code=4403&city_name=%E6%B7%B1%E5%9C%B3&lat=0.000000&lng=0.000000&network=wifi&location_city_name=",
    'host': "basesg.teld.cn",
    'content-type': "application/x-www-form-urlencoded",
    'cache-control': "no-cache",
    'postman-token': "4dd0a429-35ad-1863-c8f5-18e0f7805f60"
    }

In [4]:
def write_error(error_log, pillcode, error_msg):
    """
        Input:
            error_log - file, write error messages into designated file
    """
    error_log.write(pillcode)
    error_log.write(str(error_msg))
    error_log.write('\n')

In [5]:
def pillState_scrap(pillcode): 
    """
        Input:
            pillCode - string, each charging pile has its own pile code
        Output:
            pill_detailedInfo - dictionary of pill state details
    """
    hdr = {
            'accept': "*/*",
            'user-agent': "Teld/3.4.0 (iPhone; iOS 10.3.3; Scale/2.00)",
            'cookie': "TELDAppID=",
            'content-length': "111",
            'connection': "keep-alive",
            'accept-encoding': "gzip, deflate",
            'device': "app_version=3.4.0&os_version=10.3.3&client=ios&device_name=iPhone 6s (A1633/A1688/A1691/A1700)&device_id=38CD294C-CC36-4998-9074-C7F798EE6087&city_code=4403&city_name=%E6%B7%B1%E5%9C%B3&lat=0.000000&lng=0.000000&network=wifi&location_city_name=",
            'host': "basesg.teld.cn",
            'content-type': "application/x-www-form-urlencoded",
            'cache-control': "no-cache",
            'postman-token': "4dd0a429-35ad-1863-c8f5-18e0f7805f60"
        }
    time_out = 200
    F = None
    proxies_pool = None
    #If encounter error, try one more request with a different proxy
    try:
        pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=time_out)
    except ValueError as e:
        if F is not None:
            write_error(F, pillcode, e)
        try:
            pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=time_out)
        except ValueError as e:
            if F is not None:
                write_error(F, pillcode, e)
            pill_detailedInfo = None
        except (requests.exceptions.ChunkedEncodingError) as e:
            if F is not None:
                write_error(F, pillcode, e)
#             else:
            pill_detailedInfo = None
    except (requests.exceptions.ChunkedEncodingError) as e:
        if F is not None:
            write_error(F, pillcode, e)
        try:
            pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=time_out)
        except ValueError as e:
            if F is not None:
                write_error(F, pillcode, e)
            pill_detailedInfo = None
        except (requests.exceptions.ChunkedEncodingError) as e:
            if F is not None:
                write_error(F, pillcode, e)
#         else:
            pill_detailedInfo = None
#     return pill_detailedInfo, proxies_pool
    return pill_detailedInfo

In [6]:
def insert_state(pill_detailedInfo):
    if pill_detailedInfo is not None:
        db_name="db_teld"
        user_name="postgres"
        password = "postgres"
        host="127.0.0.1"
        port="5432"
        conn = indb.init_db(db_name, user_name, password, host, port)
        pillCode = str(pill_detailedInfo['pillCode'])
        stateCode = str(pill_detailedInfo['stateCode'])
        stateCodeV3 = str(pill_detailedInfo['stateCodeV3'])
        if pill_detailedInfo['stateName'] is not None:
            stateName = str(pill_detailedInfo['stateName'].encode('utf-8'))
        else:
            stateName = ''
        if pill_detailedInfo['stateNameV3'] is not None:
            stateNameV3 = str(pill_detailedInfo['stateNameV3'].encode('utf-8'))
        else:
            stateNameV3 = ''
        conn = Insert_py.insert_state_info(conn, pillCode, stateCode, stateName, stateCodeV3, stateNameV3)
        conn.close()

In [7]:
# ##------- Every once a while runs it update station and pile fixed info -------##
# #--------------------------------------------#
# #------- Gather all stations  -------# ****NEED TO BE FIXED 09.17.2017
# station_info = Get_py.get_allStations(hdr)
# print(len(station_info))
# for station in station_info:
#     stationId = str(station['id'])  
# #     print(stationId)
#     #------- Gather individual stations details -------#
#     station_detailedInfo = Get_py.get_oneStation_APP(hdr, stationId)
#     conn = Insert_py.insert_singStation_details(conn, station_detailedInfo)
#     #------- Gather individual pile code -------#
#     piles_info = Get_py.get_piles(stationId, proxies_pool, hdr)
#     conn = Insert_py.insert_pilecode(conn, stationId, piles_info)
# #------- Gather inidividual pile details -------#
# #------- Request stationid which has user activities from db -------#
try:
    cur = conn.cursor()
    cur.execute("select distinct stationid from tbl_app_useractivity;")
    stationWithAct_list = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
print(len(stationWithAct_list))
# #------- Request pilecode from db for states -------#
staid_list = []
for staid in stationWithAct_list:
    staid_list.append(staid[0])
try:
    cur = conn.cursor()
    stmt = "select pilecode from tbl_APP_pilecode where staid in %s"
    cur.execute(stmt, (tuple(staid_list),))
#     cur.execute("select pilecode from tbl_APP_pilecode")
    pillcode_list = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
print(len(pillcode_list))

# # ------- Requests stationID from db for user charging activities -------#
try:
    cur = conn.cursor()
    cur.execute("select stationid from tbl_app_eachstations;")
    station_list = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
print(len(station_list))

# reportTime = datetime.datetime.now() + datetime.timedelta(minutes=15)
# i = 0
# for pile in pilecode_list:
#     #If encounter error, try one more request with a different proxy
#     try:
#         pile_detailedInfo, proxies_pool = Get_py.get_onePile_APP(F, hdr, pile[0], proxies_pool, time_out=1000)
#     except ValueError as e:
#         try:
#             pile_detailedInfo, proxies_pool = Get_py.get_onePile_APP(F, hdr, pile[0], proxies_pool, time_out=1000)
#         except ValueError as e:
#             pile_detailedInfo = None
#         except (requests.exceptions.ChunkedEncodingError) as e:
#             pile_detailedInfo = None
#     except (requests.exceptions.ChunkedEncodingError) as e:
#         try:
#             pile_detailedInfo, proxies_pool = Get_py.get_onePile_APP(F, hdr, pile[0], proxies_pool, time_out=1000)
#         except ValueError as e:
#             pile_detailedInfo = None
#         except (requests.exceptions.ChunkedEncodingError) as e:
#             pile_detailedInfo = None
#     if pile_detailedInfo is not None:
#         conn = Insert_py.insert_singPile_details(conn, pile_detailedInfo)
#     else:
#         print("Running None Func.")
#         conn = Insert_py.insert_singPile_none(conn, pile_detailedInfo, pile[0])
#     i += 1
#     if datetime.datetime.now() >= reportTime:
#         reportTime = datetime.datetime.now() + datetime.timedelta(minutes=15) 
#         print("Successfully run {} piles".format(i))
# #211 Running None Func.
# #71585 piles.

3128
34473
8866


In [8]:
## ----- Test for single looping of multiprocessing post_request and insert -----##

pid = os.getpid()
py = psutil.Process(pid)
memoryUse = py.memory_info()[0]/2.**20 #in MB
print('Before runnning script, memory use:', memoryUse, 'MB')
# loop_num = 1
# max_proc =10000
pool = Pool(200)
t = time.time()
if __name__ == '__main__':
    results = [pool.apply_async(pillState_scrap, (pillcode[0],)) for pillcode in pillcode_list]
    print(len(results))
    outputs = [p.get() for p in results]
print("success, time costs is " + str(time.time()-t)) 

pid = os.getpid()
py = psutil.Process(pid)
memoryUse = py.memory_info()[0]/2.**20 #in MB
print('After runnning script,memory use:', memoryUse, 'MB')

('Before runnning script, memory use:', 67.296875, 'MB')
34473
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed

UnboundLocalError: local variable 'pill_detailedInfo' referenced before assignment

failed
failed
failed
failed
failed
failed
failed
failed
Timeout occured
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
Timeout occured
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
Timeout occured
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed


In [9]:
pillcode_list[0][0]


'1101160075101'

In [11]:
for pill_detailedInfo in outputs:
    if pill_detailedInfo is not None:
            pillCode = str(pill_detailedInfo['pillCode'])
            stateCode = str(pill_detailedInfo['stateCode'])
            stateCodeV3 = str(pill_detailedInfo['stateCodeV3'])
            if pill_detailedInfo['stateName'] is not None:
                stateName = str(pill_detailedInfo['stateName'].encode('utf-8'))
            else:
                stateName = ''
            if pill_detailedInfo['stateNameV3'] is not None:
                stateNameV3 = str(pill_detailedInfo['stateNameV3'].encode('utf-8'))
            else:
                stateNameV3 = ''
            conn = Insert_py.insert_state_info(conn, pillCode, stateCode, stateName, stateCodeV3, stateNameV3)

KeyboardInterrupt: 

In [ ]:
pid = os.getpid()
py = psutil.Process(pid)
memoryUse = py.memory_info()[0]/2.**20 #in MB
print('Before runnning script, memory use:', memoryUse, 'MB')
pool = Pool(100)
t = time.time()
if __name__ == '__main__':
    results = [pool.apply_async(insert_state, (pill_detailedInfo,)) for pill_detailedInfo in outputs]
    print(len(results))
    outputs = [p.get() for p in results]
print("success, time costs is " + str(time.time()-t)) 
pid = os.getpid()
py = psutil.Process(pid)
memoryUse = py.memory_info()[0]/2.**20 #in MB
print('After runnning script, memory use:', memoryUse, 'MB')